In [23]:
#student_name:YonghengHou
#student_number:5556661
#login:yh790

import numpy as np 

#intizize all element to zero,mean no links between them         
w, h = 4772, 4772;       
Matrix = [[0 for x in range(w)] for y in range(h)]
#read file from "e" first
with open('gr0.epa.txt','r') as f:
    for line in f:
        split=line.split(" ",2)
        #print(split[0])
        if(split[0]=='e'):
            #print(split[1])
            first=int(split[1])
            second=int(split[2])
            #because each other links
            Matrix[first][second]=1
            Matrix[second][first]=1


In [24]:
#convert to numpy array 
np_Matrix = np.array(Matrix)
print(type(Matrix[0]))

<class 'list'>


In [25]:
from scipy.sparse import csc_matrix

def pageRank(G, s = .85, maxerr = .0001):
    """
    Computes the pagerank for each of the n states
    Parameters
    ----------
    G: matrix representing state transitions
       Gij is a binary value representing a transition from state i to j.
    s: probability of following a transition. 1-s probability of teleporting
       to another state.
    maxerr: if the sum of pageranks between iterations is bellow this we will
            have converged.
    """
    n = G.shape[0]

    # transform G into markov matrix A
    A = csc_matrix(G,dtype=np.float)
    rsums = np.array(A.sum(1))[:,0]
    ri, ci = A.nonzero()
    A.data /= rsums[ri]

    # bool array of sink states
    sink = rsums==0

    # Compute pagerank r until we converge
    ro, r = np.zeros(n), np.ones(n)
    while np.sum(np.abs(r-ro)) > maxerr:
        ro = r.copy()
        # calculate each pagerank at a time
        for i in range(0,n):
            # inlinks of state i
            Ai = np.array(A[:,i].todense())[:,0]
            # account for sink states
            Di = sink / float(n)
            # account for teleportation to state i
            Ei = np.ones(n) / float(n)

            r[i] = ro.dot( Ai*s + Di*s + Ei*(1-s) )

    # return normalized pagerank
    return r/float(sum(r))




if __name__=='__main__':
    # Example extracted from 'Introduction to Information Retrieval'
    G = np_Matrix 
    print (pageRank(G,s=.86))

[2.30354239e-04 2.30354239e-04 2.30354239e-04 ... 3.22495942e-05
 3.22495942e-05 2.30354239e-04]


In [26]:
import os
import sys
import math

import numpy
import pandas
"""
Power Iteration Method
•Given a web graph (represented as M) with nnodes, where the nodes are pages and edges are hyperlinks
•Power iteration: a simple iterative scheme
• Suppose there are N web pages
• Initialize: r(0) = [1/N,….,1/N]T
• Iterate: r(t+1) = M · r(t)
• Stop when |r(t+1) – r(t)|1 < e
"""
def __extractNodes(matrix):
    nodes = set()
    for colKey in matrix:
        nodes.add(colKey)
    for rowKey in matrix.T:
        nodes.add(rowKey)
    return nodes

def __makeSquare(matrix, keys, default=0.0):
    matrix = matrix.copy()
    
    def insertMissingColumns(matrix):
        for key in keys:
            if not key in matrix:
                matrix[key] = pandas.Series(default, index=matrix.index)
        return matrix

    matrix = insertMissingColumns(matrix) # insert missing columns
    matrix = insertMissingColumns(matrix.T).T # insert missing rows

    return matrix.fillna(default)

def __ensureRowsPositive(matrix):
    matrix = matrix.T
    for colKey in matrix:
        if matrix[colKey].sum() == 0.0:
            matrix[colKey] = pandas.Series(numpy.ones(len(matrix[colKey])), index=matrix.index)
    return matrix.T

def __normalizeRows(matrix):
    return matrix.div(matrix.sum(axis=1), axis=0)

def __euclideanNorm(series):
    return math.sqrt(series.dot(series))

# PageRank specific functionality:

def __startState(nodes):
    if len(nodes) == 0: raise ValueError("There must be at least one node.")
    startProb = 1.0 / float(len(nodes))
    return pandas.Series({node : startProb for node in nodes})

def __integrateRandomSurfer(nodes, transitionProbabilities, rsp):
    alpha = 1.0 / float(len(nodes)) * rsp
    return transitionProbabilities.copy().multiply(1.0 - rsp) + alpha

def powerIteration(transitionWeights, rsp=0.15, epsilon=0.00001, maxIterations=1000):
    # Clerical work:
    transitionWeights = pandas.DataFrame(transitionWeights)
    nodes = __extractNodes(transitionWeights)
    transitionWeights = __makeSquare(transitionWeights, nodes, default=0.0)
    transitionWeights = __ensureRowsPositive(transitionWeights)

    # Setup:
    state = __startState(nodes)
    transitionProbabilities = __normalizeRows(transitionWeights)
    transitionProbabilities = __integrateRandomSurfer(nodes, transitionProbabilities, rsp)
    
    # Power iteration:
    for iteration in range(maxIterations):
        oldState = state.copy()
        state = state.dot(transitionProbabilities)
        delta = state - oldState
        if __euclideanNorm(delta) < epsilon: 
            break

    return state

powered=powerIteration(G)



In [5]:
print(powered)
print(type(powered))

0       0.001154
1       0.001941
2       0.000230
3       0.000682
4       0.000449
5       0.000316
6       0.001051
7       0.010864
8       0.000680
9       0.000920
10      0.000428
11      0.004873
12      0.006821
13      0.000787
14      0.000393
15      0.001682
16      0.000256
17      0.000337
18      0.000680
19      0.001082
20      0.003824
21      0.000462
22      0.001246
23      0.001399
24      0.000035
25      0.000441
26      0.001225
27      0.001546
28      0.000572
29      0.003561
          ...   
4742    0.000085
4743    0.000120
4744    0.000085
4745    0.000321
4746    0.000085
4747    0.000085
4748    0.000211
4749    0.000243
4750    0.000253
4751    0.000110
4752    0.000074
4753    0.000074
4754    0.000107
4755    0.000074
4756    0.000074
4757    0.000074
4758    0.000106
4759    0.000112
4760    0.000486
4761    0.000074
4762    0.000074
4763    0.000201
4764    0.000412
4765    0.000609
4766    0.000091
4767    0.000091
4768    0.000209
4769    0.0000

In [28]:
from pyspark.mllib.linalg.distributed import *
from pyspark.mllib.linalg import *
from pyspark import SparkContext
from pyspark.sql import SparkSession

import numpy as np
from pyspark.sql import Row

#sc = SparkContext("local", "PageRank")

a=powered
b=[[a[i]] for i in range(len(a))]
print(b)

[[0.0011535759207019854], [0.0019414447032429145], [0.00023008869919353828], [0.0006816185759672841], [0.00044949852603557214], [0.00031566445775230135], [0.0010511245978403832], [0.010864119858309785], [0.0006802908703937773], [0.0009199967353389273], [0.00042807213225388275], [0.004872701949029973], [0.0068210792183355925], [0.0007867860082295521], [0.00039256687670615413], [0.0016818861136035986], [0.00025551437689741034], [0.00033653382777536564], [0.0006797875221637802], [0.0010820394833984902], [0.0038241776100891526], [0.00046175599905902767], [0.0012459398512295855], [0.0013986892722779203], [3.45133048790311e-05], [0.0004406891905119204], [0.0012254299430658686], [0.001546497572560971], [0.0005718924977455135], [0.003561422934953753], [0.0006896486049632589], [0.00030870437529867807], [0.0012585768040663276], [0.00011432063990252415], [0.0026028110649044056], [0.002179795938873339], [0.007444934912029382], [0.00016140921203348062], [0.007195089652721699], [0.000528026889607916

In [ ]:
rdd = sc.parallelize(b)

print(rdd.take(10))
#rdd = spark.createDataFrame(rdd)


rdd.cache()
cordmat = CoordinateMatrix(rdd.map(lambda x: MatrixEntry(*x)))
print(cordmat.entries.collect())

#set 4x4
blockmat = cordmat.toBlockMatrix(4,4) #dimen. of blockmatrix
blockmat.cache()
print(blockmat.blocks.collect())

rdd1 = blockmat.blocks.map(lambda x: (x[0], x[1].toArray()))
print(rdd1.collect())
